# Power Profile Computation

This tutorial demonstrates how to compute the Power Profile of a Satellite orbiting the Earth using a simple power model.

## Setup

In [9]:
import datetime
import math

import numpy as np
import pandas as pd

import plotly.graph_objs as go

from IPython.display import display
import ipywidgets as widgets

from ostk.mathematics.objects import RealInterval

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.units import Derived
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate.spherical import AER
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE
from ostk.astrodynamics import Access
from ostk.astrodynamics.access import Generator as AccessGenerator

---

## Configuration

Define a timestamp (ms):

In [10]:
start_instant = Instant.date_time(DateTime.parse('2021-07-15T10:00:00.000', DateTime.Format.ISO8601), Scale.UTC)

Define an analysis duration:

In [11]:
analysis_duration = Duration.minutes(90.0)

In [12]:
step = Duration.seconds(60.0)

Define a TLE:

In [13]:
tle = TLE(
    '1 99994U          21182.53438657  .00000932  00000-0  53335-4 0 00005',
    '2 99994 097.5092 311.3101 0011006 216.6384 244.4941 15.12686890000118'
)

Define Ground Station of Interest

In [14]:
ground_station_chosen = 'Ground Station 1'

### Accesses

Define ground station data for computing accesses:

In [15]:
ground_stations = {
    'Ground Station 1': {
        'identifier': 'GS1',
        'latitude': 78.23164,
        'longitude': 15.37725,
        'altitude': 483.0,
        'min_elevation_angle': 5.0,
    },
    'Ground Station 2': {
        'identifier': 'GS2',
        'latitude': -72.0021,
        'longitude': 2.5251,
        'altitude': 1401.0,
        'min_elevation_angle': 5.0,
    },
}

---

## Orbital Elements

In [16]:
print(tle)

-- Two-Line Elements -------------------------------------------------------------------------------
    Line 1:                                  1 99994U          21182.53438657  .00000932  00000-0  53335-4 0 00005 
    Line 2:                                  2 99994 097.5092 311.3101 0011006 216.6384 244.4941 15.12686890000118 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
    Satellite Name:                                                                   
    Satellite Number:                        99994                                    
    Classification:                          U                                        
    International Designator:                                                         
    Epoch:                                   2021-07-01 12:49:30.999.648 [UTC]        
    Mean Motion First Time Der. / 2:         9.3200000000000006e-06                   
    Mean Motion Second Time Der. / 6:       

---

## Computation

In [17]:
def lla_from_state (state):
    
    lla = lla_from_position(state.get_position(), state.get_instant())

    return [
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters())
    ]

def lla_from_position (position, instant):
    
    return LLA.cartesian(
        position.in_frame(Frame.ITRF(), instant).get_coordinates(),
        Earth.equatorial_radius,
        Earth.flattening
    )

def position_from_lla (lla):
    
    return Position.meters(
        lla.to_cartesian(Earth.equatorial_radius, Earth.flattening),
        Frame.ITRF()
    )

def parse_ground_station_lla (ground_station):
    
    return LLA(
        Angle.degrees(ground_station['latitude']),
        Angle.degrees(ground_station['longitude']),
        Length.meters(ground_station['altitude'])
    )

def compute_aer (instant, from_position, to_position):
    
    from_lla = lla_from_position(from_position, instant)
    
    ned_frame = earth.get_frame_at(from_lla, Earth.FrameType.NED)

    from_position_NED = from_position.in_frame(ned_frame, instant)
    to_position_NED = to_position.in_frame(ned_frame, instant)

    aer = AER.from_position_to_position(from_position_NED, to_position_NED, True)
            
    return [
        float(aer.get_azimuth().in_degrees()),
        float(aer.get_elevation().in_degrees()),
        float(aer.get_range().in_meters())
    ]

def compute_time_lla_aer_state (state, ground_station_position):
    
    instant = state.get_instant()
    
    lla = lla_from_state(state)
    aer = compute_aer(instant, ground_station_position, state.get_position().in_frame(Frame.ITRF(), state.get_instant()))

    return [instant, lla[0], lla[1], lla[2], aer[0], aer[1], aer[2]]

def compute_trajectory_geometry (trajectory, interval):

    return [
        lla_from_state(state)
        for state in trajectory.get_states_at(interval.generate_grid(Duration.minutes(1.0)))
    ]

def compute_access_geometry (access, ground_station):
    
    ground_station_position = position_from_lla(parse_ground_station_lla(ground_station))

    return [
        compute_time_lla_aer_state(state, ground_station_position)
        for state in orbit.get_states_at(access.get_interval().generate_grid(Duration.seconds(1.0)))
    ]

def compute_eclipse (state, level: int = 1):
    
    sun = environment.access_celestial_object_with_name("Sun")
    R_earth = float(Earth.equatorial_radius.in_meters())
    R_sun = float(sun.get_equatorial_radius().in_meters())

    instant = state.get_instant()

    state.in_frame(Frame.ITRF())
    P_sat = state.get_position().get_coordinates()
    D_sat = np.linalg.norm(P_sat)

    sun.set_instant(instant)
    P_sun = sun.get_position_in(Frame.ITRF()).get_coordinates()
    D_sun = np.linalg.norm(P_sun)
    
    alpha = math.pi - math.acos(np.inner(P_sat, P_sun) / (D_sat * D_sun))
    alpha_penumbra = math.pi - math.acos((R_sun + R_earth) / D_sun) - math.acos(R_earth / D_sat)
    alpha_umbra = math.pi - math.acos((R_sun - R_earth) / D_sun) - math.acos(R_earth / D_sat)

    if level == 1:
        eclipse = alpha < alpha_penumbra
        
    else:
        if alpha < alpha_umbra:
            eclipse = "Umbra" 
            
        elif alpha < alpha_penumbra:
            eclipse = "Penumbra"
            
        else:
            eclipse = "None"
            
    return eclipse

Bootstrap environment:

In [18]:
environment = Environment.default()
earth = environment.access_celestial_object_with_name('Earth')

Setup an SGP4 model and an orbit:

In [19]:
epoch = start_instant

sgp4_model = SGP4(tle)
orbit = Orbit(sgp4_model, earth)

Now that the orbit is set, we can compute the satellite position:

In [20]:
interval = Interval.closed(
    start_instant,
    start_instant + analysis_duration
)

Generate a time grid:

In [21]:
instants = interval.generate_grid(step)

Compute satellite orbit:

In [22]:
states = [[instant, orbit.get_state_at(instant)] for instant in instants]

In [23]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)
    
    return [
        repr(instant),
        float(instant.get_modified_julian_date(Scale.UTC)),
        *state.get_position().get_coordinates().transpose().tolist(),
        *state.get_velocity().get_coordinates().transpose().tolist(),
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters()),
        compute_eclipse(state, level = 2)
    ]

In [24]:
orbit_data = [convert_state(instant, state) for [instant, state] in states]

In [25]:
orbit_df = pd.DataFrame(
    orbit_data,
    columns = [
        '$Time^{UTC}$',
        '$MJD^{UTC}$',
        '$x_{x}^{ECI}$',
        '$x_{y}^{ECI}$',
        '$x_{z}^{ECI}$',
        '$v_{x}^{ECI}$',
        '$v_{y}^{ECI}$',
        '$v_{z}^{ECI}$',
        '$Latitude$',
        '$Longitude$',
        '$Altitude$',
        '$Eclipse$'
    ]
)

Compute ground stations accesses and off ground station angular profiles:

In [26]:
def compute_sun_direction_vector (state):

    sun = environment.access_celestial_object_with_name('Sun')

    sat_position_GCRF = state.in_frame(Frame.GCRF()).get_position().get_coordinates()
    sun_position_GCRF = sun.get_position_in(Frame.GCRF()).get_coordinates()
    
    sat_to_sun_dir_GCRF = (sun_position_GCRF - sat_position_GCRF)
    sat_to_sun_dir_GCRF /= np.linalg.norm(sat_to_sun_dir_GCRF)
    
    return sat_to_sun_dir_GCRF

def compute_nadir_vector (state):

    lla = LLA.cartesian(
        state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(),
        Earth.equatorial_radius,
        Earth.flattening
    )

    ned_frame = earth.get_frame_at(lla, Earth.FrameType.NED)

    nadir_GCRF = ned_frame.get_axes_in(Frame.GCRF(), state.get_instant()).z()
    
    return nadir_GCRF

def compute_ground_station_direction_vector (state, to_position_ITRF):

    sat_position_GCRF = state.in_frame(Frame.GCRF()).get_position().get_coordinates()
    ground_station_GCRF = to_position_ITRF.in_frame(Frame.GCRF(), state.get_instant()).get_coordinates()
    
    sat_to_ground_station_dir_GCRF = (ground_station_GCRF - sat_position_GCRF)
    sat_to_ground_station_dir_GCRF /= np.linalg.norm(sat_to_ground_station_dir_GCRF)
    
    return sat_to_ground_station_dir_GCRF

def compute_body_frame_axes (state):
    
    # Compute +Y = Sun direction vector
    sun_direction_vector = compute_sun_direction_vector(state)
    y_B = sun_direction_vector

    # Compute Nadir
    nadir_vector = compute_nadir_vector(state)
    
    # Compute Nadir x y_B
    a = np.cross(nadir_vector, y_B)
    
    # Compute +X: x_B = y_B x (Nadir x y_B)
    x_B = np.cross(y_B, a) / np.linalg.norm(np.cross(y_B, a))
    
    z_B = np.cross(x_B, y_B) / np.linalg.norm(np.cross(x_B, y_B))
    
    return (x_B, y_B, z_B)

def compute_off_to_angle (state, to_position_ITRF):
    
    '''
    Compute off ground station angle (angle between antenna on satellite +X deck and ground station) at a given orbit state.
    
    This assumes Sun tracking +Y / Nadir clocking +X (YAM-3 safe mode).
    '''
    
    (x_B, y_B, z_B) = compute_body_frame_axes(state)
    
    ground_station_direction_vector = compute_ground_station_direction_vector(state, to_position_ITRF)

    return (
        np.arccos(np.dot(x_B, ground_station_direction_vector)) * 180.0 / np.pi,
        x_B
    )

def compute_ground_intersection (state, x_axis_GCRF):

    from ostk.physics.environment.object import Geometry
    from ostk.mathematics.geometry.d3.objects import Ray
    from ostk.mathematics.geometry.d3.objects import Point as Point3d

    earth_geometry = earth.get_geometry_in(Frame.GCRF())

    sat_position_GCRF = state.get_position().in_frame(Frame.GCRF(), state.get_instant()).get_coordinates()
    sat_position_point_GCRF = Point3d(sat_position_GCRF[0], sat_position_GCRF[1], sat_position_GCRF[2])

    ray = Ray(sat_position_point_GCRF, x_axis_GCRF)
    
    intersection = ray.intersection_with_ellipsoid(earth_geometry.access_composite().as_ellipsoid())
    intersection_composite = intersection.access_composite()

    intersection_point = None
    
    if intersection_composite.is_point_set():
        intersection_point = intersection_composite.as_point_set().get_point_closest_to(sat_position_point_GCRF)
    
    elif intersection_composite.is_point():
        intersection_point = intersection_composite.as_point()
    
    else:
        return None
    
    assert intersection_point is not None
    
    intersection_position_GCRF = Position.meters(
        [intersection_point.x(), intersection_point.y(), intersection_point.z()],
        Frame.GCRF()
    )
    
    intersection_lla = LLA.cartesian(
        intersection_position_GCRF.in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(),
        Earth.equatorial_radius,
        Earth.flattening
    )
    
    return [
        float(intersection_lla.get_latitude().in_degrees()),
        float(intersection_lla.get_longitude().in_degrees())
    ]

---

## Visualization (Heliocentric Reference Frame to add)

TBA

---